In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import random
import matplotlib
import matplotlib.pyplot as plt
import time
import random

from sklearn.metrics import confusion_matrix, plot_confusion_matrix, accuracy_score, plot_roc_curve,\
                             precision_recall_curve, plot_precision_recall_curve, f1_score, average_precision_score,\
                             hinge_loss, precision_score, recall_score, classification_report
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate, KFold
from sklearn.metrics import make_scorer, accuracy_score, average_precision_score, f1_score,\
                            log_loss, precision_score, recall_score, roc_auc_score
from sklearn.preprocessing import label_binarize, LabelBinarizer, LabelEncoder, OneHotEncoder
from sklearn.datasets import fetch_openml
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.utils import resample

import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import RMSprop
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import to_categorical

from catboost import CatBoostClassifier

Using TensorFlow backend.
C:\Users\Andrzej\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Andrzej\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Andrzej\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Andrzej\anaconda3\lib\site-packages\tensorflow\python\framework\d

# Data load

In [2]:
fmnist = fetch_openml("Fashion-MNIST", data_home="./fmnist", cache=True)
classes = [str(x) for x in range(0, 10)]
num_classes = len(classes)

def mk_dataset(total, fmnist=fmnist, classes=classes):
    samples = int(fmnist.data.shape[0]*total)
    return resample(fmnist.data, fmnist.target, n_samples=samples)

def plot_imgs(x, y, w=28, h=28):
    plt.figure(figsize=(10,10))
    for i in range(min(25, x.shape[0])):
        plt.subplot(5,5,i+1)
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        img = x[i]
        img = img.reshape((w, h))
        plt.imshow(img)
        plt.xlabel(y[i])
    plt.show()    

# Model evaluation

In [10]:
def binarized_scorer(metric, **kwargs):
    lb = LabelBinarizer()
    def score(y_test, y_pred, metric=metric, lb=lb, kwargs=kwargs):
        lb.fit(y_test)
        y_test = lb.transform(y_test)
        y_pred = lb.transform(y_pred)
        return metric(y_test, y_pred, **kwargs)
    return score

def mk_test(clf, name, mangling=0.0):
    def run_test(X, Y, clf=clf, name=name, mangling=mangling):
        scoring = {
            "accuracy":     binarized_scorer(accuracy_score), 
            "f1_score":     binarized_scorer(f1_score, average='macro'), 
            "log_loss":     binarized_scorer(log_loss), 
            "precision":    binarized_scorer(precision_score, average='macro'), 
            "recall":       binarized_scorer(recall_score, average='macro'), 
            "roc_auc":      binarized_scorer(roc_auc_score, average='macro'),
            # to je pole pod Precision-Recall, albo jakaś średnia. nie wiem.
            "prec_rec_auc": binarized_scorer(average_precision_score, average='macro') 
        }
        
        '''
        # Ta implementacja jest potwornie wolna dla modeli szkolonych na CPU
        # ale przynajmniej pozwala na wprowadzenie szumu do danych treningowych
        scores = {}
        Y_org = np.array(Y, copy=True)
        for train_index, test_index in KFold(n_splits=5).split(X):
            Y = np.array(Y_org, copy=True)
            to_mangle = int(len(train_index)*mangling)
            for idx in train_index[:to_mangle]:
                Y[idx] = random.choice(classes)
            clf.fit(X[train_index], Y[train_index])
            Y_true = Y[test_index]
            Y_pred = clf.predict(X[test_index])
            for (key, scorer) in scoring.items():
                if key in scores:
                    scores[key] += [scorer(Y_true, Y_pred)]
                else:
                    scores[key] = [scorer(Y_true, Y_pred)]
        '''
        
        # Bez kross-walidacji.
        scores = {}
        x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)
        to_mangle = int(len(x_train)*mangling)
        for idx in range(to_mangle):
            y_train[idx] = random.choice(classes)
        clf.fit(x_train, y_train)
        y_pred = clf.predict(x_test)
        for (key, scorer) in scoring.items():
            scores[key] = [scorer(y_test, y_pred)]
    
        scores = {k: (1.0*sum(v))/len(v) for k, v in scores.items()}
        df = pd.DataFrame(scores, index=[0])
        df.insert(loc=0, column='Name', value=name)
        return df
    return run_test

# Models

In [49]:
def mk_adaboost(depth=5, n=100, seed=1):
    return AdaBoostClassifier(
        base_estimator=DecisionTreeClassifier(max_depth=depth),
        n_estimators=n,
        random_state=seed)

def mk_catboost():
    return CatBoostClassifier(iterations=1000, task_type="GPU")

class MyLittleKerasClassifier(KerasClassifier):
    # predict() nie zwracal 
    def predict(self, X):
        y_pred = KerasClassifier.predict(self, X)
        return to_categorical(y_pred, num_classes)

def mk_mlp(epochs=10):
    def build():
        model = Sequential()
        model.add(Dense(64, activation='relu', input_shape=(28*28,)))
        model.add(Dropout(0.2))
        model.add(Dense(128, activation='relu'))
        model.add(Dropout(0.2))
        model.add(Dense(256, activation='relu'))
        model.add(Dropout(0.2))
        model.add(Dense(512, activation='relu'))
        model.add(Dropout(0.2))
        model.add(Dense(256, activation='relu'))
        model.add(Dropout(0.2))
        model.add(Dense(num_classes, activation='softmax'))
        model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
        return model
    build().summary()
    return MyLittleKerasClassifier(build_fn=build, epochs=epochs)

def mk_cnn(epochs=10):
    def build():
        model = Sequential()
        model.add(Conv2D(filters=32, kernel_size=(3, 3), input_shape=(28,28, 1), activation='relu'))
        model.add(BatchNormalization())
        model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu'))
        model.add(BatchNormalization())
        model.add(Conv2D(32, kernel_size = 5, strides=2, padding='same', activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))
        model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
        model.add(BatchNormalization())
        model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
        model.add(BatchNormalization())
        model.add(Conv2D(64, kernel_size = 5, strides=2, padding='same', activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))
        model.add(Conv2D(128, kernel_size = 4, activation='relu'))
        model.add(BatchNormalization())
        model.add(Flatten())
        model.add(Dropout(0.4))
        model.add(Dense(10, activation='softmax'))
        model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
        return model
    build().summary()
    return MyLittleKerasClassifier(build_fn=build, epochs=epochs)

# Pomiar metryk w zależności od ilości dostępnych danych

In [52]:
def test_cnn_data(data_sz):
    name = 'CNN | {}% of data'.format(data_sz*100.0)
    x, y = mk_dataset(data_sz)
    y = to_categorical(y, num_classes)
    x /= 255.0
    x = x.reshape((x.shape[0], 28, 28, 1))
    return mk_test(mk_cnn(epochs=10), name)(x, y)

def test_mlp_data(data_sz):
    name = 'MLP | {}% of data'.format(data_sz*100.0)
    x, y = mk_dataset(data_sz)
    y = to_categorical(y, num_classes)
    x /= 255.0
    return mk_test(mk_mlp(epochs=10), name)(x, y)

# Pomiar z użyciem time.time() to nie jest czas CPU
# ale już tam nic to. timeit() było zbyt irytujące
def test_adaboost_data(data_sz):
    print("Ada SZ: %d" % data_sz)
    name = 'AdaBoost | {}% of data'.format(data_sz*100.0)
    return mk_test(mk_adaboost(n=100), name)(*mk_dataset(data_sz))

In [53]:
tests_data = pd.concat([method(data_sz) for method in [test_adaboost_data, test_mlp_data, test_cnn_data] for data_sz in [0.1, 0.3, 0.5, 0.6, 0.8, 1.0]])
tests_data

Ada SZ: 0
Ada SZ: 0
Ada SZ: 0
Ada SZ: 0
Ada SZ: 0
Ada SZ: 1
Model: "sequential_55"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_199 (Dense)            (None, 64)                50240     
_________________________________________________________________
dropout_220 (Dropout)        (None, 64)                0         
_________________________________________________________________
dense_200 (Dense)            (None, 128)               8320      
_________________________________________________________________
dropout_221 (Dropout)        (None, 128)               0         
_________________________________________________________________
dense_201 (Dense)            (None, 256)               33024     
_________________________________________________________________
dropout_222 (Dropout)        (None, 256)               0         
___________________________________________________________

28000/28000 [==============================] - 5s 175us/step - loss: 0.0752 - accuracy: 0.9703
Epoch 7/10
28000/28000 [==============================] - 5s 175us/step - loss: 0.0732 - accuracy: 0.9710
Epoch 8/10
28000/28000 [==============================] - 5s 175us/step - loss: 0.0723 - accuracy: 0.9715
Epoch 9/10
28000/28000 [==============================] - 5s 177us/step - loss: 0.0702 - accuracy: 0.9720
Epoch 10/10
28000/28000 [==============================] - 5s 175us/step - loss: 0.0697 - accuracy: 0.9721
Model: "sequential_61"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_235 (Dense)            (None, 64)                50240     
_________________________________________________________________
dropout_250 (Dropout)        (None, 64)                0         
_________________________________________________________________
dense_236 (Dense)            (None, 128)               8320  

56000/56000 [==============================] - 11s 189us/step - loss: 0.0858 - accuracy: 0.9665
Epoch 3/10
56000/56000 [==============================] - 11s 188us/step - loss: 0.0789 - accuracy: 0.9690
Epoch 4/10
56000/56000 [==============================] - 11s 188us/step - loss: 0.0755 - accuracy: 0.9706
Epoch 5/10
56000/56000 [==============================] - 11s 189us/step - loss: 0.0731 - accuracy: 0.9714
Epoch 6/10
56000/56000 [==============================] - 10s 187us/step - loss: 0.0706 - accuracy: 0.9723
Epoch 7/10
56000/56000 [==============================] - 11s 189us/step - loss: 0.0690 - accuracy: 0.9727
Epoch 8/10
56000/56000 [==============================] - 11s 189us/step - loss: 0.0680 - accuracy: 0.9734
Epoch 9/10
56000/56000 [==============================] - 11s 190us/step - loss: 0.0666 - accuracy: 0.9739
Epoch 10/10
56000/56000 [==============================] - 11s 191us/step - loss: 0.0649 - accuracy: 0.9745
Model: "sequential_67"
________________________

16800/16800 [==============================] - 6s 370us/step - loss: 0.0620 - accuracy: 0.9760
Epoch 6/10
16800/16800 [==============================] - 6s 372us/step - loss: 0.0572 - accuracy: 0.9777
Epoch 7/10
16800/16800 [==============================] - 6s 360us/step - loss: 0.0541 - accuracy: 0.9787
Epoch 8/10
16800/16800 [==============================] - 6s 359us/step - loss: 0.0515 - accuracy: 0.9795
Epoch 9/10
16800/16800 [==============================] - 6s 366us/step - loss: 0.0498 - accuracy: 0.9804
Epoch 10/10
16800/16800 [==============================] - 6s 353us/step - loss: 0.0471 - accuracy: 0.9814
Model: "sequential_71"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_204 (Conv2D)          (None, 26, 26, 32)        320       
_________________________________________________________________
batch_normalization_204 (Bat (None, 26, 26, 32)        128       
_____________________

33600/33600 [==============================] - 12s 364us/step - loss: 0.0420 - accuracy: 0.9837
Epoch 9/10
33600/33600 [==============================] - 12s 364us/step - loss: 0.0396 - accuracy: 0.9848
Epoch 10/10
33600/33600 [==============================] - 12s 368us/step - loss: 0.0371 - accuracy: 0.9857
Model: "sequential_75"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_232 (Conv2D)          (None, 26, 26, 32)        320       
_________________________________________________________________
batch_normalization_232 (Bat (None, 26, 26, 32)        128       
_________________________________________________________________
conv2d_233 (Conv2D)          (None, 24, 24, 32)        9248      
_________________________________________________________________
batch_normalization_233 (Bat (None, 24, 24, 32)        128       
_________________________________________________________________
conv2d

,Name,accuracy,f1_score,log_loss,precision,recall,roc_auc,prec_rec_auc
0,AdaBoost | 10.0% of data,0.739286,0.748010,9.004752,0.753317,0.747010,0.858962,0.618273
0,AdaBoost | 30.0% of data,0.746190,0.748015,8.766270,0.752440,0.745637,0.858730,0.624200
0,AdaBoost | 50.0% of data,0.710286,0.718291,10.006377,0.728679,0.715963,0.841832,0.583490
0,AdaBoost | 60.0% of data,0.716548,0.719405,9.790098,0.732555,0.717996,0.843254,0.586704
0,AdaBoost | 80.0% of data,0.737321,0.741090,9.072596,0.745271,0.739609,0.855196,0.612381
0,AdaBoost | 100.0% of data,0.690214,0.693554,10.699620,0.709862,0.690326,0.827950,0.552017
0,MLP | 10.0% of data,0.845000,0.841632,5.353510,0.843769,0.845330,0.914048,0.742177
0,MLP | 30.0% of data,0.860952,0.860672,4.802535,0.862481,0.860372,0.922443,0.769402
0,MLP | 50.0% of data,0.867571,0.864721,4.573921,0.867263,0.867241,0.926259,0.773778
0,MLP | 60.0% of data,0.871071,0.869019,4.453035,0.873707,0.870241,0.927957,0.782170


In [54]:
tests_data.to_pickle('3aa.pkl')

# Pomiar metryk w zależności od czasów treningu

In [57]:
def test_cnn_time(epochs, data_sz=0.5):
    name = 'CNN | %d epochs' % epochs
    x, y = mk_dataset(data_sz)
    y = to_categorical(y, num_classes)
    x /= 255.0
    x = x.reshape((x.shape[0], 28, 28, 1))
    return mk_test(mk_cnn(epochs=epochs), name)(x, y)

def test_mlp_time(epochs, data_sz=0.5):
    name = 'MLP | %d epochs' % epochs
    x, y = mk_dataset(data_sz)
    y = to_categorical(y, num_classes)
    x /= 255.0
    return mk_test(mk_mlp(epochs=epochs), name)(x, y)

# Pomiar z użyciem time.time() to nie jest czas CPU
# ale już tam nic to. timeit() było zbyt irytujące
def test_adaboost_time(epochs, data_sz=0.5):
    print("AdaBoost %d" % epochs)
    start_time = time.time()
    name = 'AdaBoost'
    result = mk_test(mk_adaboost(n=epochs), name)(*mk_dataset(data_sz))
    elapsed_time = time.time() - start_time
    result['Name'] += ' | %d secs' % (elapsed_time)
    return result

## Dla Adaboosta - czas rzeczywisty

In [58]:
tests_time = [test_adaboost_time(n) for n in [10, 25, 50, 150, 500]]

AdaBoost 10
AdaBoost 25
AdaBoost 50
AdaBoost 150
AdaBoost 500


## Dla sieci - liczba epok

In [59]:
tests_time += [method(epochs) for method in [test_mlp_time, test_cnn_time] for epochs in [1, 3, 5, 15, 25, 35]]
tests_time

Model: "sequential_79"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_283 (Dense)            (None, 64)                50240     
_________________________________________________________________
dropout_316 (Dropout)        (None, 64)                0         
_________________________________________________________________
dense_284 (Dense)            (None, 128)               8320      
_________________________________________________________________
dropout_317 (Dropout)        (None, 128)               0         
_________________________________________________________________
dense_285 (Dense)            (None, 256)               33024     
_________________________________________________________________
dropout_318 (Dropout)        (None, 256)               0         
_________________________________________________________________
dense_286 (Dense)            (None, 512)             

28000/28000 [==============================] - 8s 284us/step - loss: 0.1267 - accuracy: 0.9488
Epoch 2/15
28000/28000 [==============================] - 7s 233us/step - loss: 0.0923 - accuracy: 0.9638
Epoch 3/15
28000/28000 [==============================] - 6s 230us/step - loss: 0.0858 - accuracy: 0.9661
Epoch 4/15
28000/28000 [==============================] - 7s 232us/step - loss: 0.0800 - accuracy: 0.9689
Epoch 5/15
28000/28000 [==============================] - 6s 229us/step - loss: 0.0776 - accuracy: 0.9697
Epoch 6/15
28000/28000 [==============================] - 6s 231us/step - loss: 0.0764 - accuracy: 0.9702
Epoch 7/15
28000/28000 [==============================] - 6s 230us/step - loss: 0.0726 - accuracy: 0.9715
Epoch 8/15
28000/28000 [==============================] - 6s 229us/step - loss: 0.0700 - accuracy: 0.9728
Epoch 9/15
28000/28000 [==============================] - 7s 233us/step - loss: 0.0701 - accuracy: 0.9726
Epoch 10/15
28000/28000 [==============================] 

28000/28000 [==============================] - 7s 235us/step - loss: 0.0825 - accuracy: 0.9680
Epoch 5/35
28000/28000 [==============================] - 7s 239us/step - loss: 0.0792 - accuracy: 0.9690
Epoch 6/35
28000/28000 [==============================] - 7s 234us/step - loss: 0.0761 - accuracy: 0.9699
Epoch 7/35
28000/28000 [==============================] - 7s 237us/step - loss: 0.0754 - accuracy: 0.9705
Epoch 8/35
28000/28000 [==============================] - 7s 236us/step - loss: 0.0722 - accuracy: 0.9714
Epoch 9/35
28000/28000 [==============================] - 7s 235us/step - loss: 0.0703 - accuracy: 0.9730
Epoch 10/35
28000/28000 [==============================] - 7s 254us/step - loss: 0.0707 - accuracy: 0.9726
Epoch 11/35
28000/28000 [==============================] - 7s 252us/step - loss: 0.0688 - accuracy: 0.9733
Epoch 12/35
28000/28000 [==============================] - 7s 255us/step - loss: 0.0674 - accuracy: 0.9736
Epoch 13/35
28000/28000 [=============================

Epoch 1/3
28000/28000 [==============================] - 14s 510us/step - loss: 0.1160 - accuracy: 0.9535
Epoch 2/3
28000/28000 [==============================] - 12s 430us/step - loss: 0.0774 - accuracy: 0.9695
Epoch 3/3
28000/28000 [==============================] - 12s 429us/step - loss: 0.0654 - accuracy: 0.9741
Model: "sequential_95"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_288 (Conv2D)          (None, 26, 26, 32)        320       
_________________________________________________________________
batch_normalization_288 (Bat (None, 26, 26, 32)        128       
_________________________________________________________________
conv2d_289 (Conv2D)          (None, 24, 24, 32)        9248      
_________________________________________________________________
batch_normalization_289 (Bat (None, 24, 24, 32)        128       
_________________________________________________________________

Model: "sequential_99"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_316 (Conv2D)          (None, 26, 26, 32)        320       
_________________________________________________________________
batch_normalization_316 (Bat (None, 26, 26, 32)        128       
_________________________________________________________________
conv2d_317 (Conv2D)          (None, 24, 24, 32)        9248      
_________________________________________________________________
batch_normalization_317 (Bat (None, 24, 24, 32)        128       
_________________________________________________________________
conv2d_318 (Conv2D)          (None, 12, 12, 32)        25632     
_________________________________________________________________
batch_normalization_318 (Bat (None, 12, 12, 32)        128       
_________________________________________________________________
dropout_400 (Dropout)        (None, 12, 12, 32)      

Epoch 1/35
28000/28000 [==============================] - 16s 586us/step - loss: 0.1160 - accuracy: 0.9545
Epoch 2/35
28000/28000 [==============================] - 14s 488us/step - loss: 0.0755 - accuracy: 0.9704
Epoch 3/35
28000/28000 [==============================] - 13s 471us/step - loss: 0.0644 - accuracy: 0.9750
Epoch 4/35
28000/28000 [==============================] - 14s 489us/step - loss: 0.0577 - accuracy: 0.9779
Epoch 5/35
28000/28000 [==============================] - 14s 498us/step - loss: 0.0543 - accuracy: 0.9789
Epoch 6/35
28000/28000 [==============================] - 14s 508us/step - loss: 0.0500 - accuracy: 0.9806
Epoch 7/35
28000/28000 [==============================] - 14s 487us/step - loss: 0.0470 - accuracy: 0.9817
Epoch 8/35
28000/28000 [==============================] - 14s 510us/step - loss: 0.0440 - accuracy: 0.9833
Epoch 9/35
28000/28000 [==============================] - 15s 523us/step - loss: 0.0413 - accuracy: 0.9841
Epoch 10/35
28000/28000 [============

[                 Name  accuracy  f1_score   log_loss  precision    recall  \
 0  AdaBoost | 41 secs  0.674143  0.660696  11.254707   0.662973  0.675028   
 
     roc_auc  prec_rec_auc  
 0  0.819431      0.517244  ,
                  Name  accuracy  f1_score   log_loss  precision    recall  \
 0  AdaBoost | 98 secs  0.673286  0.671653  11.284312   0.682087  0.675956   
 
     roc_auc  prec_rec_auc  
 0  0.819818      0.536792  ,
                   Name  accuracy  f1_score   log_loss  precision    recall  \
 0  AdaBoost | 194 secs  0.708143  0.708621  10.080389   0.720281  0.704137   
 
    roc_auc  prec_rec_auc  
 0   0.8359      0.571904  ,
                   Name  accuracy  f1_score  log_loss  precision    recall  \
 0  AdaBoost | 576 secs  0.730286  0.733719  9.315601   0.754045  0.728994   
 
     roc_auc  prec_rec_auc  
 0  0.849529      0.610027  ,
                    Name  accuracy  f1_score  log_loss  precision    recall  \
 0  AdaBoost | 1628 secs  0.771571  0.771118  7.88964

In [61]:
df = pd.concat(tests_time)
df.to_pickle('3bb.pkl')
df

,Name,accuracy,f1_score,log_loss,precision,recall,roc_auc,prec_rec_auc
0,AdaBoost | 41 secs,0.674143,0.660696,11.254707,0.662973,0.675028,0.819431,0.517244
0,AdaBoost | 98 secs,0.673286,0.671653,11.284312,0.682087,0.675956,0.819818,0.536792
0,AdaBoost | 194 secs,0.708143,0.708621,10.080389,0.720281,0.704137,0.835900,0.571904
0,AdaBoost | 576 secs,0.730286,0.733719,9.315601,0.754045,0.728994,0.849529,0.610027
0,AdaBoost | 1628 secs,0.771571,0.771118,7.889643,0.775195,0.771171,0.872892,0.648503
0,MLP | 1 epochs,0.814857,0.807750,6.394608,0.817601,0.816511,0.897969,0.701238
0,MLP | 3 epochs,0.866429,0.866259,4.613394,0.867767,0.866591,0.925871,0.774929
0,MLP | 5 epochs,0.855000,0.851723,5.008123,0.853429,0.855015,0.919452,0.754478
0,MLP | 15 epochs,0.875286,0.873481,4.307479,0.876812,0.875326,0.930732,0.788716
0,MLP | 25 epochs,0.881714,0.880659,4.085444,0.881988,0.880490,0.933678,0.798737


# Moc klasyfiaktora
Tutaj trzeba przemieszać pewien % etykiet zbioru treningowego.

In [29]:
def test_cnn_rob(mangling, data_sz=0.5, epochs=10):
    name = 'CNN | {}% mangling'.format(mangling*100.0)
    x, y = mk_dataset(data_sz)
    y = to_categorical(y, num_classes)
    x /= 255.0
    x = x.reshape((x.shape[0], 28, 28, 1))
    return mk_test(mk_cnn(epochs=epochs), name, mangling=mangling)(x, y)

def test_mlp_rob(mangling, data_sz=0.5, epochs=10):
    name = 'MLP | {}% mangling'.format(mangling*100.0)
    x, y = mk_dataset(data_sz)
    y = to_categorical(y, num_classes)
    x /= 255.0
    return mk_test(mk_mlp(epochs=epochs), name, mangling=mangling)(x, y)

def test_adaboost_rob(mangling, data_sz=0.5, n=100):
    name = 'AdaBoost | {}% mangling'.format(mangling*100.0)
    result = mk_test(mk_adaboost(n=n), name, mangling=mangling)(*mk_dataset(data_sz))
    return result

In [51]:
#robustness_test = [method(man) for method in [test_adaboost_rob, test_mlp_rob, test_cnn_rob] for man in [0.01, 0.05, 0.15, 0.3]]
robustness_test = [method(man) for method in [test_mlp_rob, test_cnn_rob] for man in [0.0, 0.01, 0.05, 0.15, 0.3, 0.5, 1.0]]
pd.concat(robustness_test)

Model: "sequential_31"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_115 (Dense)            (None, 64)                50240     
_________________________________________________________________
dropout_124 (Dropout)        (None, 64)                0         
_________________________________________________________________
dense_116 (Dense)            (None, 128)               8320      
_________________________________________________________________
dropout_125 (Dropout)        (None, 128)               0         
_________________________________________________________________
dense_117 (Dense)            (None, 256)               33024     
_________________________________________________________________
dropout_126 (Dropout)        (None, 256)               0         
_________________________________________________________________
dense_118 (Dense)            (None, 512)             

28000/28000 [==============================] - 4s 142us/step - loss: 1.7688 - accuracy: 0.7802
Epoch 8/10
28000/28000 [==============================] - 4s 154us/step - loss: 1.7677 - accuracy: 0.7802
Epoch 9/10
28000/28000 [==============================] - 5s 167us/step - loss: 1.7678 - accuracy: 0.7803
Epoch 10/10
28000/28000 [==============================] - 5s 165us/step - loss: 1.7680 - accuracy: 0.7803
Model: "sequential_37"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_151 (Dense)            (None, 64)                50240     
_________________________________________________________________
dropout_154 (Dropout)        (None, 64)                0         
_________________________________________________________________
dense_152 (Dense)            (None, 128)               8320      
_________________________________________________________________
dropout_155 (Dropout)        (None,

28000/28000 [==============================] - 4s 151us/step - loss: 10.1125 - accuracy: 0.0972
Epoch 4/10
28000/28000 [==============================] - 4s 158us/step - loss: 10.1125 - accuracy: 0.0972
Epoch 5/10
28000/28000 [==============================] - 4s 155us/step - loss: 10.1125 - accuracy: 0.0972
Epoch 6/10
28000/28000 [==============================] - 5s 169us/step - loss: 10.1125 - accuracy: 0.0972
Epoch 7/10
28000/28000 [==============================] - 5s 164us/step - loss: 10.1125 - accuracy: 0.0972
Epoch 8/10
28000/28000 [==============================] - 5s 162us/step - loss: 10.1125 - accuracy: 0.0972
Epoch 9/10
28000/28000 [==============================] - 5s 165us/step - loss: 10.1125 - accuracy: 0.0972
Epoch 10/10
28000/28000 [==============================] - 4s 150us/step - loss: 10.1125 - accuracy: 0.0972


C:\Users\Andrzej\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Model: "sequential_43"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_92 (Conv2D)           (None, 26, 26, 32)        320       
_________________________________________________________________
batch_normalization_92 (Batc (None, 26, 26, 32)        128       
_________________________________________________________________
conv2d_93 (Conv2D)           (None, 24, 24, 32)        9248      
_________________________________________________________________
batch_normalization_93 (Batc (None, 24, 24, 32)        128       
_________________________________________________________________
conv2d_94 (Conv2D)           (None, 12, 12, 32)        25632     
_________________________________________________________________
batch_normalization_94 (Batc (None, 12, 12, 32)        128       
_________________________________________________________________
dropout_184 (Dropout)        (None, 12, 12, 32)      

Epoch 1/10
28000/28000 [==============================] - 10s 355us/step - loss: 2.0212 - accuracy: 0.7757
Epoch 2/10
28000/28000 [==============================] - 9s 308us/step - loss: 1.8201 - accuracy: 0.7795
Epoch 3/10
28000/28000 [==============================] - 9s 310us/step - loss: 1.7764 - accuracy: 0.7798
Epoch 4/10
28000/28000 [==============================] - 8s 303us/step - loss: 1.7607 - accuracy: 0.7797
Epoch 5/10
28000/28000 [==============================] - 9s 306us/step - loss: 1.7566 - accuracy: 0.7797
Epoch 6/10
28000/28000 [==============================] - 9s 313us/step - loss: 1.7551 - accuracy: 0.7797
Epoch 7/10
28000/28000 [==============================] - 9s 315us/step - loss: 1.7545 - accuracy: 0.7797
Epoch 8/10
28000/28000 [==============================] - 9s 330us/step - loss: 1.7539 - accuracy: 0.7797
Epoch 9/10
28000/28000 [==============================] - 9s 311us/step - loss: 1.7538 - accuracy: 0.7796
Epoch 10/10
28000/28000 [====================

28000/28000 [==============================] - 10s 347us/step - loss: 5.1805 - accuracy: 0.4990
Epoch 5/10
28000/28000 [==============================] - 9s 339us/step - loss: 5.1793 - accuracy: 0.4990
Epoch 6/10
28000/28000 [==============================] - 10s 351us/step - loss: 5.1793 - accuracy: 0.4990
Epoch 7/10
28000/28000 [==============================] - 10s 354us/step - loss: 5.1798 - accuracy: 0.4990
Epoch 8/10
28000/28000 [==============================] - 9s 334us/step - loss: 5.1798 - accuracy: 0.4990
Epoch 9/10
28000/28000 [==============================] - 9s 329us/step - loss: 5.1795 - accuracy: 0.4990
Epoch 10/10
28000/28000 [==============================] - 9s 328us/step - loss: 5.1790 - accuracy: 0.4990
Model: "sequential_53"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_162 (Conv2D)          (None, 26, 26, 32)        320       
____________________________________________

C:\Users\Andrzej\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,Name,accuracy,f1_score,log_loss,precision,recall,roc_auc,prec_rec_auc
0,MLP | 1.0% mangling,0.727714,0.706931,9.404415,0.735001,0.726581,0.848189,0.594932
0,MLP | 5.0% mangling,0.690714,0.675783,10.682350,0.739160,0.688164,0.826941,0.533126
0,MLP | 15.0% mangling,0.776429,0.747428,7.721884,0.737148,0.775088,0.875103,0.662083
0,MLP | 30.0% mangling,0.726429,0.694929,9.448822,0.743245,0.729286,0.849453,0.600243
0,MLP | 50.0% mangling,0.774000,0.759196,7.805763,0.777235,0.773175,0.874046,0.651529
0,MLP | 100.0% mangling,0.097857,0.017827,31.158910,0.009786,0.100000,0.500000,0.100000
0,CNN | 1.0% mangling,0.862571,0.859230,4.746615,0.862418,0.861175,0.922965,0.766735
0,CNN | 5.0% mangling,0.829714,0.818981,5.881460,0.833432,0.828163,0.904632,0.718065
0,CNN | 15.0% mangling,0.803571,0.793650,6.784403,0.807451,0.806404,0.892313,0.685420
0,CNN | 30.0% mangling,0.807429,0.805027,6.651182,0.810626,0.808755,0.893671,0.690830


In [62]:
df = pd.conrobustness_test

[                  Name  accuracy  f1_score  log_loss  precision    recall  \
 0  MLP | 1.0% mangling  0.727714  0.706931  9.404415   0.735001  0.726581   
 
     roc_auc  prec_rec_auc  
 0  0.848189      0.594932  ,
                   Name  accuracy  f1_score  log_loss  precision    recall  \
 0  MLP | 5.0% mangling  0.690714  0.675783  10.68235    0.73916  0.688164   
 
     roc_auc  prec_rec_auc  
 0  0.826941      0.533126  ,
                    Name  accuracy  f1_score  log_loss  precision    recall  \
 0  MLP | 15.0% mangling  0.776429  0.747428  7.721884   0.737148  0.775088   
 
     roc_auc  prec_rec_auc  
 0  0.875103      0.662083  ,
                    Name  accuracy  f1_score  log_loss  precision    recall  \
 0  MLP | 30.0% mangling  0.726429  0.694929  9.448822   0.743245  0.729286   
 
     roc_auc  prec_rec_auc  
 0  0.849453      0.600243  ,
                    Name  accuracy  f1_score  log_loss  precision    recall  \
 0  MLP | 50.0% mangling     0.774  0.759196  7.8